**Note**
1. To reproduce images provided in our submission, the ideal runtime enviornment for this Jupyter notebook is Google Colab using an A100 GPU.

In [1]:
#@title Create the folders
JUPYTER_MODE = True # False if running scripts on cluster
import os
folder_paths = ['cache/EMNIST', 'out', 'src'] # create folders if they don't exist yet
for path in folder_paths:
    os.makedirs(path, exist_ok=True)
%%capture
!pip install git+https://github.com/hosford42/EMNIST.git # version with bug fixed

In [ ]:
#@title import libraries, set random seed
import torch; import torch.nn as nn; from torch.optim import Adam, SGD; from torchvision.utils import make_grid
import torch.nn.functional as F; import torchvision; from tqdm.notebook import trange
import numpy as np; import os; import matplotlib.pyplot as plt
from emnist import list_datasets, extract_training_samples, extract_test_samples
import functools; import argparse; import copy
from src.training import ExponentialMovingAverage, EarlyStopper
from src.datasets import EMNISTDataset, prepare_data_digits, split_dataset
from src.model_EMNIST import ScoreNet, generic_train, find_test_loss, \
    loss_fn, fusion_loss_fn
from src.samplers import marginal_prob_std, diffusion_coeff, \
    Euler_Maruyama_sampler_HD, ode_sampler_HD, Net
from src.plotter import plot_loss

seed = 1
print('Random seed is:', seed, '\n'); seed_directory = f'cache/EMNIST/s{seed}'
os.makedirs(seed_directory, exist_ok=True)
torch.manual_seed(seed); torch.cuda.manual_seed(seed) # seed the PyTorch RNG

sigma =  25.0; device = 'cuda'
marginal_prob_std_fn = functools.partial(marginal_prob_std, sigma=sigma, device = device)
diffusion_coeff_fn = functools.partial(diffusion_coeff, sigma=sigma, device = device)

# Define training parameters

In [ ]:
#@title dataset and training parameters
digits = [7, 9]

# target training
TRAIN_TAR_SIZE = 2**6; VAL_TAR_SIZE = 2**4
target_info = {
    'weights': [0.6, 0.4], 'train_size': TRAIN_TAR_SIZE, 'train_batch': 2**6,
    'val_size': VAL_TAR_SIZE,'val_batch': 2**4,
    'test_size': 5000,'test_batch': 200
}

# auxiliary data
TRAIN_SIZE = 25000; VAL_SIZE = 5000; TEST_SIZE = 5000 # auxiliary training sizes
auxs_info = [
        {'weights': [0.1, 0.9],'train_size': TRAIN_SIZE,'train_batch': 200,'val_size': VAL_SIZE,'val_batch': 200,
        'test_size': TEST_SIZE,'test_batch': 200},
        {'weights': [0.3, 0.7],'train_size': TRAIN_SIZE,'train_batch': 200,'val_size': VAL_SIZE,'val_batch': 200,
        'test_size': TEST_SIZE,'test_batch': 200},
        {'weights': [0.7, 0.3],'train_size': TRAIN_SIZE,'train_batch': 200,'val_size': VAL_SIZE,'val_batch': 200,
        'test_size': TEST_SIZE,'test_batch': 200},
        {'weights': [0.9, 0.1],'train_size': TRAIN_SIZE,'train_batch': 200,'val_size': VAL_SIZE,'val_batch': 200,
        'test_size': TEST_SIZE,'test_batch': 200}]

x_images, x_labels = extract_training_samples('byclass') # train
z_images, z_labels = extract_test_samples('byclass') # test
# Auxiliary data prep - clumsy but works
d0_data = EMNISTDataset(x_images, x_labels, [digits[0]]); d0_data.shuffle()
d0_train, d0_val = split_dataset(d0_data, VAL_SIZE)
d1_data = EMNISTDataset(x_images, x_labels, [digits[1]]); d1_data.shuffle()
d1_train, d1_val = split_dataset(d1_data, VAL_SIZE)
emnistdata_train, emnistdata_val = EMNISTDataset(x_images, x_labels, digits), EMNISTDataset(x_images, x_labels, digits)
emnistdata_train.data, emnistdata_val.data = torch.cat([d0_train.data, d1_train.data], dim = 0), \
    torch.cat([d0_val.data, d1_val.data], dim = 0)
emnistdata_train.targets, emnistdata_val.targets = torch.cat([d0_train.targets, d1_train.targets], dim = 0), \
    torch.cat([d0_val.targets, d1_val.targets], dim = 0)
emnistdata_train.shuffle(); emnistdata_val.shuffle()
train_aux_loaders,train_aux_datas = prepare_data_digits(emnistdata_train, digits, auxs_info, 'train')
val_aux_loaders,val_aux_datas = prepare_data_digits(emnistdata_val, digits, auxs_info, 'val')
emnistdata_test = EMNISTDataset(z_images, z_labels, digits)
test_aux_loaders,test_aux_datas = prepare_data_digits(emnistdata_test, digits, auxs_info, 'test')

# target_data prep
emnistdata_train.shuffle(); emnistdata_val.shuffle()
[train_tar_loader], [train_tar_data] = prepare_data_digits(emnistdata_train, digits, [target_info], 'train')
[val_tar_loader], [val_tar_data] = prepare_data_digits(emnistdata_val, digits, [target_info], 'val')
[test_tar_loader], [test_tar_data] = prepare_data_digits(emnistdata_test, digits, [target_info], 'test')

print(x_images.shape, x_labels.shape)
print([train_aux_loaders[i].dataset.data.shape for i in range(4)])
print([val_aux_loaders[i].dataset.data.shape for i in range(4)])
print([test_aux_loaders[i].dataset.data.shape for i in range(4)])
print(train_tar_loader.dataset.data.shape)
print(val_tar_loader.dataset.data.shape)
print(test_tar_loader.dataset.data.shape)

# Training

In [ ]:
#@title Train auxiliary score models
# auxiliary training params
N_EPOCHS_AUX = 1000
aux_train_params_list = [
    {   'name': 'HD aux 0',
        'n_epochs': N_EPOCHS_AUX, 'lr': 2e-4,
        'patience': 50, 'max_fraction': 0.5, 'decay_points': [-1],
        'ckpt_path': seed_directory + '/aux_hd_0.pth'
    },
    {   'name': 'HD aux 1',
        'n_epochs': N_EPOCHS_AUX, 'lr': 2e-4,
        'patience': 50, 'max_fraction': 0.5, 'decay_points': [-1],
        'ckpt_path': seed_directory + '/aux_hd_1.pth'
    },
    {   'name': 'HD aux 2',
        'n_epochs': N_EPOCHS_AUX, 'lr': 2e-4,
        'patience': 50, 'max_fraction': 0.5, 'decay_points': [-1],
        'ckpt_path': seed_directory + '/aux_hd_2.pth'
    },
    {   'name': 'HD aux 3',
        'n_epochs': N_EPOCHS_AUX, 'lr': 2e-4,
        'patience': 50, 'max_fraction': 0.5, 'decay_points': [-1],
        'ckpt_path': seed_directory + '/aux_hd_3.pth'
    }]
train_indices = [0,1,2,3] # aux models to train

for i, (train_aux_loader, val_aux_loader, test_aux_loader, train_params) in enumerate(
    zip(train_aux_loaders, val_aux_loaders, test_aux_loaders, aux_train_params_list)):
    print(f'\n-----Component model {i}-----')
    if i not in train_indices: # only train select component models
        print('Skipped...')
        continue
    print('Training...')
    model, train_losses, val_losses, ema_losses, last_saved_epoch = \
        generic_train(JUPYTER_MODE, train_aux_loader, val_aux_loader, train_params)
    print('Testing...')
    find_test_loss(model, test_aux_loader, text = f'HD aux {i}', num_repeats = 5)
    print('Plotting...')
    plot_loss(train_losses, val_losses, ema_losses, last_saved_epoch,
              offset = int(last_saved_epoch/10),
              text = train_params['name'], save_path = train_params['ckpt_path'][:-4] + '_loss.png')

In [ ]:
#@title baseline model
N_EPOCHS_TAR = 2000
tar_train_params = {
    'name': 'baseline',
    'n_epochs': N_EPOCHS_TAR, 'lr': 2e-4,
    'patience': 50, 'max_fraction': 0.5, 'decay_points': [-1],
    'ckpt_path': seed_directory + f'/base_hd_{TRAIN_TAR_SIZE}.pth'
}

baseline, base_train_losses, base_val_losses, base_ema_losses, base_last_saved_epoch = \
    generic_train(JUPYTER_MODE, train_tar_loader, val_tar_loader, tar_train_params)
print('Testing...')
find_test_loss(baseline, test_tar_loader, text = f'HD base {TRAIN_TAR_SIZE}', num_repeats = 5)
print('Plotting...')
plot_loss(base_train_losses, base_val_losses, base_ema_losses, base_last_saved_epoch,
            offset = int(base_last_saved_epoch/5),
            text = tar_train_params['name'], save_path = seed_directory + f'/base_hd_{i}' + '_loss.png')

## Run sampling experiments (and train the fusion model)

In [ ]:
#@title Load auxiliary models, store as a list, and sample

device = 'cuda' #param ['cuda', 'cpu'] {'type':'string'}
comp_samples = []
comp_score_models = nn.ModuleList() # store the score models

for d in range(4): # the auxiliary digits
    ckpt = torch.load(f'cache/EMNIST/aux_hd_{d}.pth', map_location=device)
    score_model = torch.nn.DataParallel(ScoreNet(marginal_prob_std=marginal_prob_std_fn))
    score_model = score_model.to(device)
    score_model.load_state_dict(ckpt)

    for param in score_model.parameters():
        param.requires_grad = False # freeze the gradients

    comp_score_models.append(score_model)
    sample_batch_size = 512 #param {'type':'integer'}
    sampler = Euler_Maruyama_sampler_HD
    samples = sampler(score_model,
                    marginal_prob_std_fn,
                    diffusion_coeff_fn,
                    sample_batch_size,
                    device=device,jupyter_mode = True)
    # samples = samples.clamp(0.0, 1.0)
    # do not clamp the samples -> no normalize for now
    comp_samples.append(samples)

In [ ]:
from torch.distributions import Categorical
torch.set_printoptions(precision=3, sci_mode = False, linewidth=150)
np.set_printoptions(precision=2, suppress=True, linewidth=80, threshold=1000)

def classify(network, i):
    samples_size = 500
    network.eval()
    with torch.no_grad():
        output = network(comp_samples[i])
        distribution = Categorical(logits=output)
        samples = distribution.sample((samples_size,))
    print(f'Auxiliary model {i}:')
    print(f'Digits count: {(torch.bincount(samples.ravel()) / (sample_batch_size * samples_size)).cpu().numpy()}')
    # breakpoint()

network = Net()
network = network.to(device)
ckpt = torch.load(f'cache/emnist_classifier.pth', map_location=device)
network.load_state_dict(ckpt)
for i in range(4):
    classify(network, i)

In [8]:
d = 2 # 72% 7, 24% 9 - the auxiliary closest to 60%, 40%
ema_decay = 0.999
countdown = 100

train_params = {
    'name': 'finetune aux',
    'n_epochs': 200, 'lr': 5e-5,
    'patience': 50, 'max_fraction': 0.5, 'decay_points': [-1],
    'ckpt_path': f'{seed_directory}/ckpt_finetune_{d}.pth'
    }

In [ ]:
n_epochs, lr, patience, max_fraction, decay_points, ckpt_path = \
    [train_params[key] for key in ['n_epochs', 'lr', 'patience', 'max_fraction', 'decay_points', 'ckpt_path']]

aux_ckpt = torch.load(f'cache/EMNIST/aux_hd_{d}.pth', map_location=device)
score = torch.nn.DataParallel(ScoreNet(marginal_prob_std=marginal_prob_std_fn)).to(device)
score = score.to(device)
score.load_state_dict(aux_ckpt)
ema_score = ExponentialMovingAverage(score.parameters(), decay = ema_decay)
optimizer = Adam(score.parameters(), lr=lr)
val_iter = iter(val_tar_loader)
train_losses, val_losses, ema_losses = [], [], []
early_stopper = EarlyStopper(patience=patience,
                                    max_fraction_over=max_fraction,
                                    decay_points=decay_points)
earlyStopTriggered, last_saved_epoch = False, -1
tqdm_epoch = trange(n_epochs)

for epoch_num, epoch in enumerate(tqdm_epoch):
    if earlyStopTriggered: # start the countdown once there is sign of overfitting
        countdown -= 1
    for x in train_tar_loader:
        x = x[0].to(device)
        loss = loss_fn(score, x, marginal_prob_std_fn)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        ema_score.update(score.parameters())
    # calculate EMA val loss
    ema_score.store(score.parameters())  # Cache non-EMA weights
    ema_score.copy_to(score.parameters())  # Replace with EMA weights
    with torch.no_grad():
        try:
            x_val = next(val_iter)
        except StopIteration: # Reinitialize the iterator and fetch the next batch
            val_iter = iter(val_tar_loader)
            x_val = next(val_iter)
        x_val = x_val[0].to(device)
        ema_val_loss = loss_fn(score, x_val, marginal_prob_std_fn)
        doStop, gotBetter = early_stopper.early_stop(ema_val_loss, epoch_num)
        if gotBetter and not earlyStopTriggered:
            torch.save(score.state_dict(), ckpt_path)
            last_saved_epoch = epoch_num
    ema_score.restore(score.parameters())
    # calculate non-ema val loss
    with torch.no_grad():
        val_loss = loss_fn(score, x_val, marginal_prob_std_fn)
    train_losses.append(loss.item()); val_losses.append(val_loss.item()); ema_losses.append(ema_val_loss.item())
    if JUPYTER_MODE:
        tqdm_epoch.set_description(f'Train Loss: {loss.item():.4f}; Val Loss: {val_loss.item():.4f}; EMA val Loss: {ema_val_loss.item():.4f}')
    if doStop and not earlyStopTriggered: # if earlystop threshold reached, then start the countdown
        earlyStopTriggered = True
    if earlyStopTriggered and countdown == 0:
        break

In [ ]:
#@title evaluate fusion performance on the test dataset
ckpt = torch.load(f'{seed_directory}/ckpt_finetune_{d}.pth', map_location=device)
score_model = torch.nn.DataParallel(ScoreNet(marginal_prob_std=marginal_prob_std_fn))
score_model = score_model.to(device)
score_model.load_state_dict(ckpt)

CALC_TEST, VISUAL = True, True
find_test_loss(score_model, test_tar_loader, f'finetune aux {d}') if CALC_TEST else None

plot_loss(train_losses, val_losses, ema_losses, last_saved_epoch,
          offset = 0, text = f'finetune aux {d}')

# Sampling & Eval

In [ ]:
#@title Sampling from Finetuned Auxiliaries

sample_batch_size = 10000 #@param {'type':'integer'}
sampler = Euler_Maruyama_sampler_HD

## Generate samples using the specified sampler.
finetune_samples = sampler(score_model,
                  marginal_prob_std_fn,
                  diffusion_coeff_fn,
                  sample_batch_size,
                  device=device,
                  jupyter_mode=JUPYTER_MODE)
# fusion_samples = fusion_samples.clamp(0.0, 1.0)
# do not clamp the data, since it prevents the classifier from working properly

#Fusion visualization.
sample_grid = make_grid(finetune_samples[:64] * 0.3081 + 0.1307, nrow=8)

plt.figure(figsize=(6,6))
plt.axis('off')
plt.imshow(sample_grid.permute(1, 2, 0).cpu(), vmin=0., vmax=1.)
plt.show()

In [ ]:
# Finetune visualization - alternative
sample_grid = make_grid(finetune_samples[320:384] * 0.3081 + 0.1307, nrow=8)

plt.figure(figsize=(6,6))
plt.axis('off')
plt.imshow(sample_grid.permute(1, 2, 0).cpu(), vmin=0., vmax=1.)
plt.show()

In [ ]:
np.set_printoptions(precision=3, suppress=True, linewidth=80, threshold=1000)

def classify_samples(network, samples, batch_size, sample_size = 512):
    network.eval()
    with torch.no_grad():
        output = network(samples)
        distribution = Categorical(logits=output)
        samples = distribution.sample((sample_size,))
        # output = network(samples)
        # pred = output.data.max(1, keepdim=True)[1]
    print(f'Digits count: {(torch.bincount(samples.ravel()) / (batch_size * sample_size)).cpu().numpy()}')
    # breakpoint()

network = Net()
network = network.to(device)
ckpt = torch.load(f'cache/emnist_classifier.pth', map_location=device)
network.load_state_dict(ckpt)
print('Finetune digits frequency')
classify_samples(network,finetune_samples, 10000)

In [ ]:
#@title Visualize true digits data.
import matplotlib.pyplot as plt
from torchvision.utils import make_grid

fig, axes = plt.subplots(2, 5, figsize=(15, 6))  # Adjust the figsize to better suit your display
fig.suptitle("Random samples from the auxiliary dataset (ground truth)", fontsize = 16)
for d in [0,1,2,3]:
    sample_grid = make_grid(digits_samples[d][:64] * 0.3081 + 0.1307, nrow=8)
    ax = axes[d // 5, d % 5]
    image_data = sample_grid.permute(1, 2, 0).cpu().numpy()
    ax.imshow(image_data, vmin=np.min(image_data),
              vmax=np.max(image_data), cmap='gray', interpolation='none')
    ax.axis('off')
    ax.set_title(f'Auxiliary ground truth {d}')
plt.tight_layout()
plt.show()

In [ ]:
#@title Auxiliary model sample visualization.
%matplotlib inline
import matplotlib.pyplot as plt

fig, axes = plt.subplots(2, 4, figsize=(12, 6))  # Adjust the figsize to better suit your display
fig.suptitle("Digits generated by auxiliary diffusion score models", fontsize = 16)
for d in range(4):
    sample_grid = make_grid(comp_samples[d][:64] * 0.3081 + 0.1307, nrow=8)
    ax = axes[d // 4, d % 4]
    ax.imshow(sample_grid.permute(1, 2, 0).cpu(), vmin=0., vmax=1.)
    ax.axis('off')
    ax.set_title(f'Digit {d}')

plt.tight_layout()
plt.show()

## Likelihood Computation

Evaluate NLL loss

In [15]:
#@title Define the likelihood function (double click to expand or collapse)
from scipy import integrate

def prior_likelihood(z, sigma):
  """The likelihood of a Gaussian distribution with mean zero and
      standard deviation sigma."""
  shape = z.shape
  N = np.prod(shape[1:])
  return -N / 2. * torch.log(2*np.pi*sigma**2) - torch.sum(z**2, dim=(1,2,3)) / (2 * sigma**2)

def ode_likelihood(x,
                   score_model,
                   marginal_prob_std,
                   diffusion_coeff,
                   batch_size=64,
                   device='cuda',
                   eps=1e-5):
  """Compute the likelihood with probability flow ODE.

  Args:
    x: Input data.
    score_model: A PyTorch model representing the score-based model.
    marginal_prob_std: A function that gives the standard deviation of the
      perturbation kernel.
    diffusion_coeff: A function that gives the diffusion coefficient of the
      forward SDE.
    batch_size: The batch size. Equals to the leading dimension of `x`.
    device: 'cuda' for evaluation on GPUs, and 'cpu' for evaluation on CPUs.
    eps: A `float` number. The smallest time step for numerical stability.

  Returns:
    z: The latent code for `x`.
    bpd: The log-likelihoods in bits/dim.
  """

  # Draw the random Gaussian sample for Skilling-Hutchinson's estimator.
  epsilon = torch.randn_like(x)

  def divergence_eval(sample, time_steps, epsilon):
    """Compute the divergence of the score-based model with Skilling-Hutchinson."""
    with torch.enable_grad():
      sample.requires_grad_(True)
      score_e = torch.sum(score_model(sample, time_steps) * epsilon)
      grad_score_e = torch.autograd.grad(score_e, sample)[0]
    return torch.sum(grad_score_e * epsilon, dim=(1, 2, 3))

  shape = x.shape

  def score_eval_wrapper(sample, time_steps):
    """A wrapper for evaluating the score-based model for the black-box ODE solver."""
    sample = torch.tensor(sample, device=device, dtype=torch.float32).reshape(shape)
    time_steps = torch.tensor(time_steps, device=device, dtype=torch.float32).reshape((sample.shape[0], ))
    with torch.no_grad():
      score = score_model(sample, time_steps)
    return score.cpu().numpy().reshape((-1,)).astype(np.float64)

  def divergence_eval_wrapper(sample, time_steps):
    """A wrapper for evaluating the divergence of score for the black-box ODE solver."""
    with torch.no_grad():
      # Obtain x(t) by solving the probability flow ODE.
      sample = torch.tensor(sample, device=device, dtype=torch.float32).reshape(shape)
      time_steps = torch.tensor(time_steps, device=device, dtype=torch.float32).reshape((sample.shape[0], ))
      # Compute likelihood.
      div = divergence_eval(sample, time_steps, epsilon)
      return div.cpu().numpy().reshape((-1,)).astype(np.float64)

  def ode_func(t, x):
    """The ODE function for the black-box solver."""
    time_steps = np.ones((shape[0],)) * t
    sample = x[:-shape[0]]
    logp = x[-shape[0]:]
    g = diffusion_coeff(torch.tensor(t)).cpu().numpy()
    sample_grad = -0.5 * g**2 * score_eval_wrapper(sample, time_steps)
    logp_grad = -0.5 * g**2 * divergence_eval_wrapper(sample, time_steps)
    return np.concatenate([sample_grad, logp_grad], axis=0)

  init = np.concatenate([x.cpu().numpy().reshape((-1,)), np.zeros((shape[0],))], axis=0)
  # Black-box ODE solver
  res = integrate.solve_ivp(ode_func, (eps, 1.), init, rtol=1e-5, atol=1e-5, method='RK45')
  zp = torch.tensor(res.y[:, -1], device=device)
  z = zp[:-shape[0]].reshape(shape)
  delta_logp = zp[-shape[0]:].reshape(shape[0])
  sigma_max = marginal_prob_std(1.)
  prior_logp = prior_likelihood(z, sigma_max)
  bpd = -(prior_logp + delta_logp) / np.log(2)
  N = np.prod(shape[1:])
  bpd = bpd / N + 8.
  return z, bpd

In [ ]:
#@title Compute likelihood on the dataset (double click to expand or collapse)
from tqdm import notebook

finetune_all_bpds = 0.
finetune_all_items = 0

# list of batch-averaged nll values - should be distributed according to the CLT
finetune_estimators = []

try:
  tqdm_data = notebook.tqdm(test_tar_loader)
  for x, _ in tqdm_data:
    x = x.to(device)
    # uniform dequantization
    x = ((((x * 0.3081 + 0.1307) * 255. + torch.rand_like(x)) / 256) - 0.1307) / 0.3081
    _, finetune_bpd = ode_likelihood(x, score_model, marginal_prob_std_fn,
                            diffusion_coeff_fn,
                            x.shape[0], device=device, eps=1e-5)

    finetune_all_bpds += finetune_bpd.sum()
    finetune_all_items += finetune_bpd.shape[0]
    finetune_estimator = finetune_bpd.sum() / finetune_bpd.shape[0]
    finetune_estimators.append(finetune_estimator)

    tqdm_data.set_description(f"finetune avg bits/dim: {finetune_all_bpds / finetune_all_items:5f}")

except KeyboardInterrupt:
  # Remove the error message when interuptted by keyboard or GUI.
  pass


In [ ]:
finetune_estimators_torch = torch.tensor(finetune_estimators)

finetune_estimators_torch.std()

# Miscellaneous

fusion on target data: 3.8

fusion on auxiliary data 0: 4.0

auxiliary 0 on auxiliary data 0: 4.45